# Testing models

In [4]:
import sys
sys.path.append('src/')

from Models import *


In [5]:
deberta_nli('a child was very sad and crying over a dropped ice cream', 'A child cried over dropped ice cream')

([['0.00023431', '0.98561573', '0.01415000']], ['entailment'])

In [3]:
bart_nli('a child was very sad and crying over a dropped ice cream', 'A child cried over dropped ice cream')

([['0.00009507', '0.00135235', '0.99855262']], ['entailment'])